In [1]:
import requests
import pandas as pd
import tqdm
from collections import defaultdict
import urllib3
import os
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# Registro Nacional de Sociedades

In [12]:
def unificar_rns(csv_files, existing = None):
    current = pd.concat([pd.read_csv(f'./input/{file}') for file in csv_files]).drop_duplicates()
    if existing is None:
        return current
    else:
        new = current.loc[~current['cuit'].isin(existing['cuit'].unique())]
        all_data = pd.concat([existing, new], ignore_index= True).drop_duplicates()
        return all_data

In [13]:
#files = os.listdir('./input/')
#csv_files = [file for file in files if file.endswith('.csv')]

In [14]:
#existing = pd.read_parquet('./input/registro_nacional_sociedades.parquet')

In [ ]:
#all_data = unificar_rns(csv_files, existing)

In [19]:
#nuevos = list(set(all_data['cuit']) - set(existing['cuit']))

In [17]:
#all_data.to_parquet('./input/registro_nacional_sociedades.parquet', compression = 'gzip')

In [18]:
#nuevos.to_parquet('./input/cuits_nuevos.parquet', compression= 'gzip')

# Deudas Históricas

In [3]:
all_data = pd.read_parquet('./input/registro_nacional_sociedades.parquet')
cuits = all_data['cuit'].unique()

In [20]:
base_url = "https://api.bcra.gob.ar/centraldedeudores/v1.0/Deudas/Historicas/"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [21]:
todas_las_filas = []

session = requests.Session()

def fetch_data(id):
	try:
		url = f"{base_url}{id}"
		res = session.get(url, verify=False)
		res.raise_for_status()
		data = res.json()
		
		results = data['results']
		identificacion_val = results.get('identificacion')
		denominacion = results.get('denominacion')
		periodos = results.get('periodos', [])
		
		filas = []
		for periodo in periodos:
			periodo_val = periodo.get('periodo')
			entidades = periodo.get('entidades', [])
			for entidad in entidades:
				fila = {'identificacion': identificacion_val,
						'denominacion': denominacion,
						'periodo': periodo_val,
						'entidad': entidad.get('entidad'),
						'situacion': entidad.get('situacion'),
						'monto': entidad.get('monto'),
						'enRevision': entidad.get('enRevision'),
						'procesoJud': entidad.get('procesoJud')}
				filas.append(fila)
		return filas, None
	except requests.exceptions.RequestException as e:
		return None, e

with ThreadPoolExecutor() as executor:
	futures = {executor.submit(fetch_data, id): id for id in cuits}
	for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
		filas, error = future.result()
		if filas:
			todas_las_filas.extend(filas)

100%|██████████| 16760/16760 [01:11<00:00, 232.78it/s]


In [22]:
columnas = ['identificacion', 'denominacion', 'periodo', 'entidad', 'situacion', 'monto', 'enRevision', 'procesoJud']
creditos_hist = pd.DataFrame(todas_las_filas, columns= columnas)

In [26]:
creditos_hist.to_parquet('./input/creditos_hist.parquet', compression= 'gzip')

# Deudas No Históricas

In [2]:
base_url = "https://api.bcra.gob.ar/centraldedeudores/v1.0/Deudas/"
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
all_data = pd.read_parquet('./input/registro_nacional_sociedades.parquet')

In [5]:
cuits = all_data['cuit'].unique()

In [6]:
session = requests.Session()

def fetch_data(id):
	try:
		url = f"{base_url}{id}"
		res = session.get(url, verify=False)
		res.raise_for_status()
		data = res.json()
		
		results = data['results']
		identificacion_val = results.get('identificacion')
		denominacion = results.get('denominacion')
		periodos = results.get('periodos', [])
		
		filas = []
		for periodo in periodos:
			periodo_val = periodo.get('periodo')
			entidades = periodo.get('entidades', [])
			for entidad in entidades:
				fila = {'identificacion': identificacion_val,
						'denominacion': denominacion,
						'periodo': periodo_val,
						'entidad': entidad.get('entidad'),
						'situacion': entidad.get('situacion'),
						'monto': entidad.get('monto'),
						'enRevision': entidad.get('enRevision'),
						'procesoJud': entidad.get('procesoJud'),
						'fechaSit1': entidad.get('fechaSit1'),
						'diasAtrasoPago': entidad.get('diasAtrasoPago'),
						'refinanciaciones': entidad.get('refinanciaciones'),
						'recategorizacionOblig': entidad.get('recategorizacionOblig'),
						'situacionJuridica': entidad.get('situacionJuridica'),
						'irrecDisposicionTecnica': entidad.get('irrecDisposicionTecnica')}
				filas.append(fila)
		return filas, None
	except requests.exceptions.RequestException as e:
		return None, (id, str(e))

todas_las_filas = []

with ThreadPoolExecutor() as executor:
	futures = {executor.submit(fetch_data, id): id for id in cuits}
	for future in tqdm.tqdm(as_completed(futures), total=len(futures)):
		filas, error = future.result()
		if filas:
			todas_las_filas.extend(filas)

100%|██████████| 1197287/1197287 [1:10:48<00:00, 281.82it/s]


In [7]:
columnas = ['identificacion', 'denominacion', 'periodo', 'entidad', 'situacion', 'monto', 'enRevision', 'procesoJud', 'fechaSit1', 'diasAtrasoPago', 'refinanciaciones', 'recategorizacionOblig', 'situacionJuridica', 'irrecDisposicionTecnica']
creditos = pd.DataFrame(todas_las_filas, columns= columnas)

In [9]:
creditos_hist = pd.read_parquet('./input/creditos_hist.parquet')
creditos_hist_2 = creditos[creditos_hist.columns]
creditos_hist = pd.concat([creditos_hist_2, creditos_hist], ignore_index= True).drop_duplicates()

In [10]:
creditos_hist.to_parquet('./input/creditos_hist.parquet', compression='gzip')

In [11]:
creditos_2 = pd.read_parquet('./input/creditos.parquet')
creditos = pd.concat([creditos_2, creditos], ignore_index= True).drop_duplicates()

In [12]:
creditos.to_parquet('./input/creditos.parquet', compression= 'gzip')

# Principales Variables Monetarias

In [2]:
url = 'https://api.bcra.gob.ar/estadisticas/v3.0/monetarias'
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
res = requests.get(url, verify= False)

In [3]:
variables = pd.DataFrame(res.json()['results'])

In [4]:
ids = [1]
ids.extend([i for i in range(4, 46, 1)])

In [5]:
session = requests.Session()
principales_variables = []

for id in ids:
    try:
        url = f'https://api.bcra.gob.ar/estadisticas/v3.0/monetarias/{id}?desde=2022-11-01&hasta=2024-12-30&limit=3000'
        res = session.get(url, verify= False)
        res.raise_for_status()
        df = pd.DataFrame(res.json()['results'])
        principales_variables.append(df)
    except Exception as e:
        continue
    
principales_variables = pd.concat(principales_variables, axis=0, ignore_index=True)

In [6]:
variables = variables[['idVariable', 'descripcion']]
principales_variables = principales_variables.merge(
    variables,  
    on='idVariable',   
    how='left'           
)

In [7]:
principales_variables = principales_variables.pivot(
    index='fecha',    
    columns='descripcion',  
    values='valor'    
)

In [8]:
principales_variables.to_parquet('./input/principales_variables.parquet', compression= 'gzip')